In [1]:
import requests
from bs4 import BeautifulSoup

In [14]:
# The user scores URL
user_scores_url = "https://musescore.com/user/19710/scores/4766391"

# Send an HTTP GET request to the user scores URL
response = requests.get(user_scores_url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find the download link for the specific score ID (4766391)
score_id = "4766391"
download_link = soup.find(f"/score/download/index?score_id={score_id}&type=mxl")

if download_link:
    # Extract the complete download URL
    complete_download_url = "https://musescore.com" + download_link["href"]
    print(f"Download URL for score ID {score_id}: {complete_download_url}")
else:
    print(f"Score with ID {score_id} not found on the page.")

Score with ID 4766391 not found on the page.


In [15]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<script>
    (function () {
        var isTouch = 'ontouchstart' in window || 'onmsgesturechange' in window;
        var isMac = ['Macintosh', 'MacIntel', 'MacPPC', 'Mac68K'].indexOf(window.navigator.platform) > -1;

        if (isMac && isTouch) {
            var date = new Date();
            date.setFullYear(date.getFullYear() + 1);

            document.cookie = '_mu_force_ipad=1;path=/;domain=.musescore.com;expires=' + date.toUTCString() + ';';
            window.location = window.location.href
        }
    })()
</script>
<meta content="musescore:score" property="og:type"/>
<meta content="Musescore.com" property="og:site_name"/>
<meta content="Gymnopédie No. 1 – Satie" property="og:title"/>
<meta content="https://musescore.com/user/19710/scores/4766391" property="og:url"/>
<meta content="Download and print in PDF or MIDI free sheet music of 3 Gymnopédies - Erik Satie for 3 Gymnopédies by Erik Satie arranged by ClassicMan for Piano (Solo)"

In [6]:
def download_musicxml(url, total_files):
    session = requests.Session()
    count = 0

    while count < total_files:
        response = session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', href=True)
        musicxml_links = [link['href'] for link in links if link['href'].endswith('.musicxml')]

        for link in musicxml_links:
            if count >= total_files:
                break

            file_response = session.get(link)
            filename = link.split('/')[-1]

            with open(filename, 'wb') as file:
                file.write(file_response.content)

            count += 1
            print(f'Downloaded {filename} ({count}/{total_files})')

        next_page = soup.find('a', {'rel': 'next'})
        if next_page:
            url = next_page['href']
        else:
            print('No more pages to download from.')
            break

In [5]:
def download_musicxml(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)

def get_musicxml_urls(base_url, number_of_files):
    urls = []
    for i in range(1, number_of_files + 1):
        page_url = f"{base_url}?page={i}"
        response = requests.get(page_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and 'download' in href and 'musicxml' in href:
                urls.append(href)
    return urls


base_url = 'https://musescore.com/sheetmusic?complexity=1&recording_type=public-domain'

musicxml_urls = get_musicxml_urls(base_url, 3)

for index, url in enumerate(musicxml_urls):
    filename = f"music_{index + 1}.xml"
    download_musicxml(url, filename)
    print(f"from: {url} downloaded: {filename}")


In [7]:
# URL to the public domain music page on musescore.com
public_domain_music_url = 'https://musescore.com/sheetmusic?complexity=1&recording_type=public-domain'

# Number of MusicXML files to download
number_of_files_to_download = 2

# Call the function to start downloading
download_musicxml(public_domain_music_url, number_of_files_to_download)

No more pages to download from.
